<a href="https://colab.research.google.com/gist/uquant77/e43117d35ad9ad7c0cc93d973066508c/lstm-ipynb-1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Load Python libraries

In [1]:
# pip install numpy, pandas, pprint
import numpy as np
import pandas as pd
import pprint
# pip install torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable 
import torch.nn.init as init

# pip install matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# pip install ccxt
!pip install ccxt
import ccxt

     |████████████████████████████████| 2.4 MB 4.8 MB/s 
     |████████████████████████████████| 1.1 MB 27.7 MB/s 
     |████████████████████████████████| 271 kB 33.7 MB/s 
     |████████████████████████████████| 3.6 MB 42.1 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 291 kB 35.4 MB/s 
     |████████████████████████████████| 144 kB 44.1 MB/s 


In [2]:
config = {
    "data": {
        "window_size": 14,
        "train_split_size": 0.5,
        "val_split_size": 0.2,
        "thres_frac": 1.2,
        "change_time": 3
    }, 
    "model": {
        "input_size": 2, # price, volume
        "num_lstm_layers": 1,
        "hidden_size": 64,
        "num_classes" : 3,
        "dropout": 0.2,
    },
    "training": {
        "device": "cpu", # "cuda" or "cpu"
        "batch_size": 32,
        "epoch": 10,
        "learning_rate": 0.01,
        "scheduler_step_size": 40,
    }
}

2. Data Preparation

(1)Fetch Data

In [3]:
binance = ccxt.binance()
fetch_num = 500
timeframe = '1d'
ticker = binance.fetch_ohlcv("BTC/USDT", timeframe, limit=fetch_num)
startfrom = ticker[0][0]
class ohlcv:
  def __init__(self, index):
    self.index = index
    since = startfrom - 1800000000 * 24 * self.index
    ohlcv = binance.fetch_ohlcv("BTC/USDT", timeframe, since=since, limit=fetch_num)
    self.df = pd.DataFrame(ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])

ohlcv_df = pd.concat([ohlcv(2).df, ohlcv(1).df, ohlcv(0).df])
ohlcv_df = ohlcv_df.drop_duplicates(['datetime'])
ohlcv_df = ohlcv_df.drop(ohlcv_df.loc[ohlcv_df['close']==0].index)
ohlcv_df = ohlcv_df.drop(ohlcv_df.loc[ohlcv_df['volume']==0].index)
ohlcv_df['p_change'] = ohlcv_df['close'] - ohlcv_df['close'].shift(1)
ohlcv_df['p_changerate'] = np.log(ohlcv_df['close']/ ohlcv_df['close'].shift(1))
ohlcv_df['v_changerate'] = np.log(ohlcv_df['volume']/ ohlcv_df['volume'].shift(1))
ohlcv_df = ohlcv_df.drop([0])
ohlcv_df['datetime'] = pd.to_datetime(ohlcv_df['datetime'], unit='ms')
datetime = ohlcv_df['datetime'].to_numpy()
ohlcv_df.set_index('datetime', inplace=True)
ohlcv_df.to_excel("ohlcv3.xlsx")
print(ohlcv_df.describe())
p_change = ohlcv_df['p_change'].to_numpy()
p_changerate = ohlcv_df['p_changerate'].to_numpy()
v_changerate = ohlcv_df['v_changerate'].to_numpy()
print(p_change.shape)

               open          high  ...  p_changerate  v_changerate
count   1497.000000   1497.000000  ...   1497.000000   1497.000000
mean   18993.546981  19541.320929  ...      0.000629     -0.001065
std    17931.469431  18449.260344  ...      0.040926      0.397943
min     3211.710000   3276.500000  ...     -0.502607     -3.687514
25%     7131.590000   7340.000000  ...     -0.016147     -0.246804
50%     9575.000000   9759.820000  ...      0.001501     -0.029903
75%    33502.870000  34717.270000  ...      0.018675      0.242643
max    67525.820000  69000.000000  ...      0.178449      2.599870

[8 rows x 8 columns]
(1497,)


# 새 섹션

(2)Data into train, test data

In [4]:
# normalize
def normalize(x):
  mu = np.average(x)
  sd = np.std(x)
  normalized_x = (x - mu) / sd
  print(mu)
  print(sd)
  return normalized_x

p_changerate = normalize(p_changerate)
v_changerate = normalize(v_changerate)

#perform windowing
def prepare_data_x(p_changerate, v_changerate, window_size):
   n_row = p_changerate.shape[0] - window_size + 1
   x1 = np.lib.stride_tricks.as_strided(p_changerate, shape=(n_row,window_size), strides=(p_changerate.strides[0],p_changerate.strides[0]))
   x2 = np.lib.stride_tricks.as_strided(v_changerate, shape=(n_row,window_size), strides=(v_changerate.strides[0],v_changerate.strides[0]))
   li = [-i for i in range(1, config["data"]["change_time"]+1)]
   x1 = np.delete(x1, li, axis=0)
   x2 = np.delete(x2, li, axis=0)
   x = np.dstack([x1, x2])
   x = x.astype(np.float32)
   return x

#calculate deviation by window
def prepare_data_y(p_change, window_size, fraction):
  n_row = p_change.shape[0] - window_size + 1
  windowed_change = np.lib.stride_tricks.as_strided(p_change, shape=(n_row,window_size), strides=(p_change.strides[0],p_change.strides[0]))
  threshold = fraction * np.std(windowed_change, axis=1)
  print(threshold.shape)
  yn = np.zeros(n_row-1)
  for i in range(n_row-1):
    li = p_change[window_size+i:window_size+i+ config["data"]["change_time"]]
    change = sum(li)
    if change >= threshold[i]:
      yn[i] = 1
    elif change <= -threshold[i]:
      yn[i] = 2
  return yn


x = prepare_data_x(p_changerate, v_changerate, config["data"]["window_size"])
y = prepare_data_y(p_change, config["data"]["window_size"],  config["data"]["thres_frac"])

# split dataset
def split_dataset(x, y, shuffle=False):
  split_index = int(x.shape[0]* config["data"]["train_split_size"])
  split_index2 = int((x.shape[0]* (config["data"]["train_split_size"]+config["data"]["val_split_size"])
  x_train = x[:split_index]
  x_val = x[split_index: split_index2]
  x_test = x[split_index2:]
  y_train = y[:split_index]
  y_val = y[split_index: split_index2]
  y_test = y[split_index2:]
  return x_train, y_train, x_val, y_val, x_test, y_test

x_train = split_dataset(x, y)[0]
y_train = split_dataset(x, y)[1]
x_val = split_dataset(x, y)[2]
y_ val = split_dataset(x, y)[3]
x_test = split_dataset(x, y)[4]
y_test = split_dataset(x, y)[5]

class TimeSeriesDataset(Dataset):
    def __init__(self, x, y):
        self.x = x.astype(np.float32)
        self.y = y.astype(np.int64)
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

dataset_train = TimeSeriesDataset(x_train, y_train)
dataset_val = TimeSeriesDataset(x_val, y_val)
dataset_test = TimeSeriesDataset(x_test, y_test)

train_loader = DataLoader(dataset_train, batch_size=config["training"]["batch_size"], shuffle=False)
val_loader = DataLoader(dataset_val, batch_size=config["training"]["batch_size"], shuffle=False)
test_loader = DataLoader(dataset_test, batch_size=config["training"]["batch_size"], shuffle=False)

count = 0
for (x_train, y_train) in train_loader:
  count += 1
print(count)
for (x_train, y_train) in train_loader: 
  print(x_train.size(), x_train.type())
  print(y_train.size(), y_train.type())
  break

0.0006292588035143804
0.040911879727100066
-0.001065308288639101
0.39781017599999197
(1484,)
33
torch.Size([32, 14, 2]) torch.FloatTensor
torch.Size([32]) torch.LongTensor


3. Defining Model

In [5]:
class LSTMModel(nn.Module):
  def __init__(self, num_classes, input_size, hidden_size, num_layers):
    super().__init__()
    self.num_classes = num_classes #number of classes
    self.num_layers = num_layers #number of layers
    self.input_size = input_size #input size
    self.hidden_size = hidden_size #hidden state
    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm1
    self.fc1 = nn.Linear(hidden_size, 512) #fully connected 1
    self.fc2 = nn.Linear(512, 128) #fully connected 2
    self.fc3 = nn.Linear(128, num_classes) #fully connected last layer
    self.relu = nn.ReLU()
    self.batch_norm1 = nn.BatchNorm1d(hidden_size)
    self.batch_norm2 = nn.BatchNorm1d(512)
    self.batch_norm3 = nn.BatchNorm1d(128)
    self.dropout_prob = 0.2
  
  def forward(self,x):
    # Propagate input through LSTM
    output, (hn, cn) = self.lstm(x) #lstm with input, hidden, and internal state
    x = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
    x = self.batch_norm1(x)
    x = self.fc1(x) #first Dense
    x = self.batch_norm2(x)
    x = self.relu(x) #relu
    x = F.dropout(x, training=self.training, p=self.dropout_prob)
    x = self.fc2(x) #second Dense
    x = self.batch_norm3(x)
    x = self.relu(x) #relu
    x = F.dropout(x, training=self.training, p=self.dropout_prob)
    x = self.fc3(x) #Final Output
    x = F.log_softmax(x)
    return x

model = LSTMModel(num_classes=config["model"]["num_classes"], input_size=config["model"]["input_size"], hidden_size=config["model"]["hidden_size"], num_layers=config["model"]["num_lstm_layers"])
model = model.to(config["training"]["device"])

4. Optimizer, Objective Function 설정하기

In [6]:
optimizer = optim.Adam(model.parameters(), lr=config["training"]["learning_rate"])
criterion = nn.CrossEntropyLoss()
def weight_init(m):
  if isinstance(m, nn.Linear):
    nn.init.kaiming_uniform(m.weight.data)
model.apply(weight_init)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  """


LSTMModel(
  (lstm): LSTM(2, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=3, bias=True)
  (relu): ReLU()
  (batch_norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

5. Train model

In [7]:
def train(model, train_loader, optimizer, log_interval = 20):
  model.train()
  for idx, (x, y) in enumerate(train_loader):
    x = x.to(config["training"]["device"])
    y = y.to(config["training"]["device"])
    optimizer.zero_grad()
    out = model(x)
    loss = criterion(out, y)
    loss.backward()
    optimizer.step()
    if idx % log_interval == 0:
      print("Train Epoch: {} [{}/{} ({:.0f}%)] Train Loss: {:.6f}".format(epoch, idx * len(x),
      len(train_loader.dataset), 100. * idx / len(train_loader), loss.item()))

6. Evaluate Model

In [8]:
def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0

  with torch.no_grad():
    for x, y in test_loader:
      x = x.to(config["training"]["device"])
      y = y.to(config["training"]["device"])
      out = model(x)
      test_loss += criterion(out, y).item()
      prediction = out.max(1, True)[1]
      correct += prediction.eq(y.view_as(prediction)).sum().item()
  
  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy, prediction, out

7. 학습 진행 및 평가

In [10]:
for epoch in range(1, config["training"]["epoch"] + 1):
  train(model, train_loader, optimizer, log_interval=10)
  test_loss, test_accuracy, prediction, out = evaluate(model, val_loader)
  print("[EPOCH: {}], Test Loss: {:.4f}, Test Accuracy: {:.2f} %".format(epoch, test_loss, test_accuracy))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/1036 (0%)] Train Loss: 0.802172
Train Epoch: 1 [320/1036 (30%)] Train Loss: 0.989690
Train Epoch: 1 [640/1036 (61%)] Train Loss: 1.185845
Train Epoch: 1 [960/1036 (91%)] Train Loss: 0.726407
[EPOCH: 1], Test Loss: 0.0380, Test Accuracy: 47.42 %
Train Epoch: 2 [0/1036 (0%)] Train Loss: 0.779618
Train Epoch: 2 [320/1036 (30%)] Train Loss: 0.826282
Train Epoch: 2 [640/1036 (61%)] Train Loss: 1.031748
Train Epoch: 2 [960/1036 (91%)] Train Loss: 0.690479
[EPOCH: 2], Test Loss: 0.0472, Test Accuracy: 45.39 %
Train Epoch: 3 [0/1036 (0%)] Train Loss: 0.734260
Train Epoch: 3 [320/1036 (30%)] Train Loss: 0.882606
Train Epoch: 3 [640/1036 (61%)] Train Loss: 0.917519
Train Epoch: 3 [960/1036 (91%)] Train Loss: 0.675098
[EPOCH: 3], Test Loss: 0.0380, Test Accuracy: 48.99 %
Train Epoch: 4 [0/1036 (0%)] Train Loss: 0.750791
Train Epoch: 4 [320/1036 (30%)] Train Loss: 0.765672
Train Epoch: 4 [640/1036 (61%)] Train Loss: 0.975580
Train Epoch: 4 [960/1036 (91%)] Train Loss: 0.616992
[E